# Chapter 2: Tests
In this chapter, we explain the main idea behind test classes `InteractiveTest`, `BatchTest` and `BatchTestWithSplit`.

## Test Class Interface
Test class interface is defined by `cognibench.testing.CNBTest` class, which defines the functions to be implemented by concrete test clasess and provides the base implementation for multi-subject testing in addition to helper functionalities such as data and log persistence.

In [1]:
from cognibench.testing import CNBTest
print('CNBTest documentation')
print('-----------------------------')
print(CNBTest.__doc__)

CNBTest documentation
-----------------------------

    Base test class for all CogniBench tests.

    This class defines the common functionality that can be used by further testing classes. In addition to sciunit
    interaction, it defines the multi-subject testing framework, and requires deriving classes to only define
    single-subject testing logic. The deriving classes should implement at least `predict_single` and `compute_score_single` methods
    to define the testing procedure. CNBTest only accepts models that are subclasses of :class:`cognibench.models.CNBModel`.
    


## Interactive Tests
`InteractiveTest` is test class which **requires** the models to implement `cognibench.capabilities.Interactive` interface. Such models should implement the methods that allow them to be trained in a continuous manner. This is in contrast to models that first must be trained, and then can only predict actions for stimuli with no further training.

In [2]:
from cognibench.testing import InteractiveTest
print('InteractiveTest documentation')
print('-----------------------------')
print(InteractiveTest.__doc__)

InteractiveTest documentation
-----------------------------

    Perform interactive tests by feeding the input samples (stimuli, rewards, actions) one at a time and updating the
    model after each sample with the corresponding reward.
    


In [3]:
# models are required to implement cognibench.capabilities.Interactive
InteractiveTest.required_capabilities

(cognibench.capabilities.Interactive,)

`InteractiveTest` class overrides the `predict_single` and `compute_score_single` methods of `cognibench.testing.CNBTest` class to test a given model interactively. It requires three sets of observations:
* stimuli
* actions
* rewards


## Batch Tests
`BatchTest` is test class which generates test predictions from a model by providing all the samples together. Since this is a very common way of testing models in general, `cognibench` offers such a test class, as well.

In [4]:
from cognibench.testing import BatchTest
print('BatchTest documentation')
print('-----------------------------')
print(BatchTest.__doc__)

BatchTest documentation
-----------------------------

    BatchTest class allows passing the stimuli-action pairs to the model in a single batch instead of
    performing interactive testing. `predict` method of a model used in this testing method must accept
    a sequence of stimuli, not just one stimulus.
    


## Batch Tests With Splitting Functionality
By default, `cognibench` testing classes uses the same samples for both model optimization and prediction generation. However, this behaviour can be easily modified by overriding `get_fitting_observations_single` and `get_testing_observations_single` methods, which is exactly what `BatchTestWithSplit` class does. If you want to use separate samples for model fitting and prediction generation, you can use `BatchTestWithSplit` class.

In [5]:
from cognibench.testing import BatchTestWithSplit
print('BatchTestWithSplit documentation')
print('-----------------------------')
print(BatchTestWithSplit.__doc__)

BatchTestWithSplit documentation
-----------------------------

    Testing class that allows specifying training and testing samples separately for each subject. This is in contrast
    to the standard :class:`CNBTest` class where models are optimized and tested on the same samples (which is common
    practice for psychophysiological models).
    


## Implementing Your Own Testing Logic
In case various testing manners provided by `cognibench` does not satisfy your requirements, you can implement your own testing logic by subclassing from `CNBTest` class and defining how a single-subject model should be tested. This has the benefit that your class will automatically have multi-subject testing functionality, as well. If you want to follow this route, please look at the implementation of the above concrete test classes which is found in `cognibench/testing/tests.py`.